In [1]:
import copy

import torch
from torchsummary import summary

import numpy as np
import pandas as pd
import random

from models import ResNet50
# from utils import get_dataset, average_weights, exp_details
# from utils_v2 import get_dataset, average_weights, exp_details
from update import LocalUpdate, test_inference

In [2]:
# parameters
iid = 0 # if the data is i.i.d or not
unbalanced = 1 # in non i.i.d. setting split the data between clients equally or not
num_users = 100 # number of client
frac = 0.1 # fraction of the clients to be used for federated updates
n_epochs = 100
gpu = 0
optimizer = "sgd" #sgd or adam
local_batch_size = 10 # batch size of local updates in each user
lr = 0.001 # learning rate
local_epochs = 1
loss_function = "CrossEntropyLoss"

num_groups = 4  # 0 for BatchNorm, > 0 for GroupNorm
if num_groups == 0:
    normalization_type = "BatchNorm"
else:
    normalization_type = "GroupNorm"

In [3]:
if iid:
    from utils_v2 import get_dataset, average_weights, weighted_average_weights, exp_details
else:
    from utils import get_dataset, average_weights, weighted_average_weights, exp_details

In [4]:
exp_details("ResNet50", optimizer, lr, normalization_type, n_epochs, iid, frac,
            local_batch_size, local_epochs, unbalanced, num_users)


Experimental details:
    Model     : ResNet50
    Optimizer : sgd
    Learning  : 0.001
    Normalization  : GroupNorm
    Global Rounds   : 100

    Federated parameters:
    Non-IID - unbalanced
    NUmber of users  : 100
    Fraction of users  : 0.1
    Local Batch size   : 10
    Local Epochs       : 1



In [5]:
# for REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)

g = torch.Generator()
g.manual_seed(0)

np.random.seed(0)

In [6]:
train_dataset, test_dataset, user_groups = get_dataset(iid=iid, unbalanced=unbalanced,
                                                       num_users=num_users)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = ResNet50(n_type=normalization_type)
# model = CNNCifar()

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    device = torch.device("cpu")
    gpu = 0
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda")
    gpu = 1

model.to(device)

# set the model to train
model.train()

CUDA is available!  Training on GPU ...


ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1),

In [12]:
clients = []

idxs_users = range(num_users)

# init all clients
for idx in idxs_users:
    clients.append(copy.deepcopy(model))


In [9]:
for epoch in range(1, n_epochs+1):

    print(f'Epoch: {epoch} \n')

    m = max(int(frac * num_users), 1) # number of users to be used for federated updates, at least 1
    idxs_chosen_users = np.random.choice(range(num_users), m, replace=False) # choose randomly m users

    # dispatch
    for idx in idxs_chosen_users:
        for old_params, new_params in zip(clients[idx].parameters(), model.parameters())
            old_params.data = new_params.data.clone()

    # clients update
    for idx in idxs_chosen_users:
        clients[idx] = train(clients[idx], model, epoch)

    # aggregation
    s = 0
    for idx in idxs_chosen_users:
        s += clients[idx].len

    params = {}
    for k, v in clients[0].named_parameters():
        params[k] = torch.zeros_like(v.data)

    for idx in idxs_chosen_users:
        for k, v in clients[idx].named_parameters():
            params[k] += v.data * (clients[idx].len / s)

    for k, v in model.named_parameters():
        v.data = params[k].data.clone()


SyntaxError: expected ':' (1735952895.py, line 10)